In [1]:
import numpy as np
import random
import time
from itertools import combinations

In [2]:
def compute_subset_bandwidth(subset):
    BANDWIDTH = 10
    n = len(subset)
    if (n==1):
        return  n*BANDWIDTH/2
    bandwidth = n*(n-1)*BANDWIDTH/2 + n*BANDWIDTH/2
    return bandwidth

In [76]:
def check_camera_coverage(solution,subsets,set_all_cameras):
    set_ = []
    
    for s in solution:
        sub = subsets[s]
        for c in sub:
            set_.append(c)
    set_ = set(set_)
    # print(set_)
    return set_==set(set_all_cameras)
# check_camera_coverage([[1,1,1],[2,2,2]],{0,1,2})

In [77]:
def compute_subset_accuracy(subset,set_all_accs):
    ## Will implement later
    n = len(subset)
    sum_ = 0
    for s in subset:
        sum_ += set_all_accs[s]
    sum_ = sum_/n
    return sum_

In [78]:
def process_data(filepath):
    
    file_ = open(filepath, "r")
    lines = file_.readlines() 
    set_all_accs =  [int(x.rstrip()) for x in lines]
    total_bandwidth = set_all_accs[0]
    set_all_accs = set_all_accs[1:]
    set_all_cameras = [i for i in range(len(set_all_accs))]
    subsets = []
    subsets_bandwidth = []
    subsets_acc = []
    for i in range(1,5):
        combs = combinations(range(len(set_all_cameras)), i)
        for subset in combs:
            subsets.append(subset)
            subsets_bandwidth.append(compute_subset_bandwidth(subset))
            subsets_acc.append(compute_subset_accuracy(subset,set_all_accs))
            

    total_num_subsets = [i for i in range(len(subsets))]
    # print(subsets)

    return total_bandwidth,total_num_subsets, set_all_cameras, subsets, subsets_acc, subsets_bandwidth



In [79]:
total_bandwidth,total_num_subsets, set_all_cameras, subsets, subsets_acc, subsets_bandwidth = process_data("input.txt")

In [80]:
total_bandwidth

10000

In [81]:
def greedy_randomized_construction(alpha, total_num_subsets, set_all_cameras):

    MAX = 1000000
    solution = set()
    #solution_camera_attributes = set() # this is actually a set of subsets or set of cameras
    solution_camera_attributes = []
    candidates = set(total_num_subsets) # this is actually a set of all the subset that we have by index
    # print(candidates)
    while not check_camera_coverage(solution, subsets,set_all_cameras):
        ratio = dict()
        for i in candidates:
            if i not in solution:
                ratio[i] = subsets_acc[i] / subsets_bandwidth[i]
        c_min = min(ratio.values())
        c_max = max(ratio.values())
        
        RCL = [i for i in candidates if i in ratio.keys() and ratio[i] <= c_min + alpha * (c_max - c_min)]
        #print(len(candidates),len(RCL))
        selected_index = random.choice(RCL)
        # take out that specific subset's index from the set of candidates (subset)
        candidates -= {selected_index}
        # add the index of the subset to the solution
        solution.add(selected_index)
        # add the subset to the solution_camera_attributes list
        solution_camera_attributes.append(subsets[selected_index]) 
    return solution
        
solution = greedy_randomized_construction(0.1, total_num_subsets, set_all_cameras)


In [82]:
solution

{56, 76, 161, 240, 292, 327, 359, 373, 375}

In [83]:
check_camera_coverage(solution,subsets,set_all_cameras)

True

In [84]:
subsets[110]

(1, 5, 7)

In [85]:
def compute_accuracy(solution):
    sum_ = 0
    for s in solution:
        sum_+=subsets_acc[s]
    return sum_

In [86]:
def compute_bandwidth(solution):
    sum_ = 0
    for s in solution:
        sum_+=subsets_bandwidth[s]
    return sum_

In [87]:
def is_feasible(solution,total_bandwidth):
    return check_camera_coverage(solution,subsets,set_all_cameras) and compute_bandwidth(solution) <= total_bandwidth

In [106]:
# check that there are no redudant cameras
def remove_redundant_cameras(solution, subsets, set_all_cameras):
    for i in solution:
        temp_sol = solution.copy()
        temp_sol.remove(i)
        if check_camera_coverage(temp_sol,subsets,set_all_cameras):
            solution = temp_sol.copy()
    return solution

In [115]:
# remove_redundant_cameras(solution,subsets,set_all_cameras)

In [116]:
# check_camera_coverage({56, 161, 327, 380},subsets,set_all_cameras)

In [117]:
# solution = {56,161,327,380}

This is the correct local search

In [113]:
def local_search(solution, subsets,set_all_cameras):

    clean_soln_set = remove_redundant_cameras(solution,subsets,set_all_cameras)

    # Subsets not in the solution
    subsets_not_in_solution = set([s for s in total_num_subsets if s not in sol_set])

    best_accuracy = compute_accuracy(solution)
    best_solution_set = solution.copy()

    # flags to decide whether to remove a subset; or swap a subset during the search    
    remove_subset_flag = True
    swap_subset_flag = True

    while remove_subset_flag or swap_subset_flag:

        # Set as false for now; until one of the situations evaluates to be true later
        remove_subset_flag = False
        swap_subset_flag = False

        temp_sol = clean_soln_set.copy()
        for i_out in clean_soln_set:

            # remove i_out index of subset if redundant
            temp_sol.difference_update({i_out})

            if check_camera_coverage(temp_sol,subsets,set_all_cameras):
                remove_subset_flag = True
                clean_soln_set.difference_update({i_out})
                # improved and exit
                break
            else:
                temp_sol.update({i_out})

            
            for i_in in subsets_not_in_solution:

                # If the subset entering the solution has higher cost than the one exiting, improvement is not possible
                # This local search never allows the solution to get worst
                if subsets_acc[i_in] >= subsets_acc[i_out]:

                    temp_sol.update({i_in})
                    temp_sol.difference_update({i_out})

                    accuracy = compute_accuracy(temp_sol)

                    if accuracy > best_accuracy and check_camera_coverage(temp_sol,subsets,set_all_cameras):
                        swap_subset_flag = True

                        best_accuracy = accuracy
                        best_solution_set = temp_sol.copy()

                        sub_in = i_in
                        sub_out = i_out

                    temp_sol = clean_soln_set.copy()
        # if a swap was undertaken, update the subsets_not_in_solution accordingly
        if swap_subset_flag:

            subsets_not_in_solution.difference_update({sub_in})
            subsets_not_in_solution.update({sub_out})

            clean_soln_set = best_solution_set.copy()
            print('Updated solution', clean_soln_set)
            print('Updated accuracy', best_accuracy)
    
    return clean_soln_set

If you run the below, you will see the accuracy go up

In [118]:
# local_search(solution, subsets, set_all_cameras)

In [119]:
# def local_search(solution):
#     # we take the current solution and the unused subsets
#     # iterate and swap if it leads to a better solution in terms of higher accuracy

#     # Subsets not in solution
#     subsets_not_in_solution = set([s for s in total_num_subsets if s not in solution])
#     print(solution)
#     # print(subsets_not_in_solution)
#     best_accuracy = compute_accuracy(solution)
#     print(best_accuracy)
#     best_solution_set = solution.copy()

#     temp_solution = solution.copy()
#     for s_out in solution:
#         for s_in in subsets_not_in_solution:
#             # recall we want higher accuracy so we swap in s_in if it has higher acc
#             if subsets_acc[s_in] > subsets_acc[s_out]:
#                 # do the swap if we can get better accuracy based on simple comparison of the 2 subsets accuracy
#                 temp_solution.update({s_in})
#                 print('Add', s_in)
#                 temp_solution.difference_update({s_out})
#                 print('Remove', s_out)
#                 accuracy = compute_accuracy(temp_solution)

#                 if accuracy > best_accuracy and is_feasible(temp_solution,total_bandwidth):
#                     # update the current best accuracy and solution
#                     best_accuracy = accuracy
#                     best_solution_set = temp_solution
#                     #print(best_accuracy,compute_bandwidth(temp_solution))
#     return best_solution_set
# best_sol = local_search(solution)

In [120]:
def repair_unfeasible(solution):
    subsets_not_in_solution = set([s for s in total_num_subsets if s not in solution])
    best_bandwidth = compute_bandwidth(solution)

    #best_bandwidth = compute_bandwidth(solution)
    #best_solution_set = solution.copy()

    temp_solution = solution.copy()

    for s_out in solution:
        subs = subsets[s_out]
        temp_solution.difference_update({s_out})
        if len(subs)>1:
            for s_in in subs: 
                temp_solution.update({s_in})
            #if bandwidth < best_bandwidth:

            if is_feasible(temp_solution,total_bandwidth):
                return temp_solution
            #solution = temp_solution
    return None

In [121]:
def GRASP(num_iterations, alpha, num_sample_pool, path_relinking_flag=True):
    best_acc = 0
    best_sol = None
    # list of solutions that we have gone through
    S = []
    iteration = num_iterations
    while iteration > 0:
        iteration -= 1
        solution = greedy_randomized_construction(alpha, total_num_subsets,set_all_cameras)
        # Change this to use the updated local search
        solution = local_search(solution, subsets, set_all_cameras)
        #print("F",solution)
        if not is_feasible(solution,total_bandwidth):
            solution = repair_unfeasible(solution)
            #print("D",solution)
        S.append(solution)
        #print("Best Accuracy: ",best_acc)
        acc = compute_accuracy(S[-1])
        if acc > best_acc:
            best_acc = acc
            best_sol = S[-1]
        print("Best Accuracy: ",best_acc, " Bandwidth: ",compute_bandwidth(best_sol))
    return best_acc, best_sol

best_acc,best_sol = GRASP(100, 0.1, 10, False)
print(best_acc,compute_bandwidth(best_sol))

Best Accuracy:  226.5  Bandwidth:  240.0
Best Accuracy:  239.0  Bandwidth:  240.0
Best Accuracy:  239.0  Bandwidth:  240.0
Best Accuracy:  239.0  Bandwidth:  240.0
Best Accuracy:  239.0  Bandwidth:  240.0
Best Accuracy:  239.0  Bandwidth:  240.0
Best Accuracy:  383.08333333333337  Bandwidth:  260.0
Best Accuracy:  383.08333333333337  Bandwidth:  260.0
Updated solution {258, 5, 327, 376, 284}
Updated accuracy 391.25
Updated solution {258, 5, 22, 327, 376}
Updated accuracy 408.5
Updated solution {5, 22, 327, 376, 217}
Updated accuracy 419.75
Best Accuracy:  383.08333333333337  Bandwidth:  260.0
Best Accuracy:  383.08333333333337  Bandwidth:  260.0
Best Accuracy:  383.08333333333337  Bandwidth:  260.0
Best Accuracy:  383.08333333333337  Bandwidth:  260.0
Best Accuracy:  383.08333333333337  Bandwidth:  260.0
Best Accuracy:  383.08333333333337  Bandwidth:  260.0
Best Accuracy:  383.08333333333337  Bandwidth:  260.0
Best Accuracy:  383.08333333333337  Bandwidth:  260.0
Best Accuracy:  383.08